# Position Reconstruction for XeBRA - Neural Networks

**Status:** August 2019

Position reconstruction for XeBRA using a feedforward neural network with Keras.
Purpose of this notebook: Keras Hyperparameter Tuning using Hyperas.

In [1]:
#############################################
## Use Google Colab:
#############################################
b_GC = False # True or False, also adjust in data()!

if b_GC == True:
    !pip install hyperas
    !pip install hyperopt
    !pip install uproot
    !pip install numpy
    !pip install pandas

In [2]:
#############################################
## Imports:
#############################################

from __future__ import print_function
import numpy as np

from hyperopt import Trials, STATUS_OK, tpe
from keras.layers.core import Dense, Dropout, Activation
from keras.models import Sequential
from keras.utils import np_utils
from keras.optimizers import SGD,Adam

from hyperas import optim
from hyperas.distributions import choice, uniform

import os
import glob
import uproot
import pandas as pd

Using TensorFlow backend.


In [3]:
if b_GC == True:
    ## Import datasets uploaded on Google Drive
    ## by mounting the Google Drive into the runtime.
    from google.colab import drive
    drive.mount('/content/gdrive')
    !ls 'gdrive/My Drive/Colab Notebooks'

In [4]:
#############################################
## Preparing the Data
#############################################

def data():
    """
    Data providing function:

    This function is separated from create_model() so that hyperopt
    won't reload data for each evaluation run.
    """

    ## Selection of file(s) to be used:
    b_GC = False # True or False
    if b_GC == True:
        filename_list = glob.glob("gdrive/My Drive/Simulation Data/optPhot_S2_1.0e+05_pmtGap4.5_RPTFE0.95_LXeAbs100_Tmesh0.89770509_writeEmptyFalse_nop_1.0e+04_*.root")
    else:
        filename_list = glob.glob("/media/ab602/TOSHIBA EXT/Simulation_Data/LCE/optPhot_S2_1.0e+05_pmtGap4.5_RPTFE0.95_LXeAbs100_Tmesh0.89770509_writeEmptyFalse_nop_1.0e+04_*.root")                
    file_split_train = 0.9 # fraction of file used for training
    
    ## Import as pandas dataframes
    branches= ['xp_pri','yp_pri','zp_pri','ntpmthits','pmthits'] # Be careful that the branches are also initilized in _chunk_collect
    import concurrent.futures
    executor = concurrent.futures.ThreadPoolExecutor(8)
    _chunk_collect = pd.DataFrame({'xp_pri' : [],'yp_pri' : [],'zp_pri' : [], 'ntpmthits' : [],'pmthits' : []}) # Initialize empty data frame
    for _chunk in uproot.iterate(filename_list, "events/events",branches, entrysteps=100000, 
                                           outputtype=pd.DataFrame, executor=executor):
        _chunk.columns=branches
        _chunk_collect=_chunk_collect.append(_chunk)

    ## Normalize to hits fractions
    _chunk_collect['pmthits'] = (_chunk_collect['pmthits'] / _chunk_collect['ntpmthits'])

    ## Separate hits for individual PMTs
    ## Takes a while to calculate...
    for i in range(1,8):
        _chunk_collect['pmthits_'+str(i)] = pd.Series([tupl[i] for tupl in np.asarray(_chunk_collect['pmthits'])], index=_chunk_collect.index)

    _chunk_collect = _chunk_collect.drop(columns=['pmthits'])
    _chunk_collect = _chunk_collect.drop(columns=['ntpmthits'])

    ## Separate labels and features
    features=['pmthits_1','pmthits_7']
    labels=['xp_pri','yp_pri']

    ## Split arrays into (random) train and test subsets.
    ## Could also be done with sklearn.model_selection.train_test_split,
    ## but already randomized from simulation (see plots bleow).
    train_label     = _chunk_collect.loc[:int(np.floor(_chunk_collect.shape[0]*file_split_train))-1,labels]
    predict_label   = _chunk_collect.loc[int(np.floor(_chunk_collect.shape[0]*file_split_train)):,labels]
    train_feature   = _chunk_collect.loc[:int(np.floor(_chunk_collect.shape[0]*file_split_train))-1,features[0]:features[1]]
    predict_feature = _chunk_collect.loc[int(np.floor(_chunk_collect.shape[0]*file_split_train)):,features[0]:features[1]]

    del(_chunk_collect)

    ## Convert to numpy arrays
    train_feature   = train_feature.to_numpy()
    train_label     = train_label.to_numpy()
    predict_feature = predict_feature.to_numpy()
    predict_label   = predict_label.to_numpy()

    ## Normalize labels to [-0.5, 0.5] to make network easier to train 
    ## as using smaller, centered values is often better.
    ## Have to rescale later.
    scaling_factor = 70
    train_label = train_label/scaling_factor
    predict_label = predict_label/scaling_factor
    
    ## Return
    return train_feature, train_label, predict_feature, predict_label

In [5]:
#############################################
## Function for optimizing the NN
#############################################

def create_model(train_feature, train_label, predict_feature, predict_label):
    """
    Model providing function:

    Create Keras model with double curly brackets dropped-in as needed.
    Return value has to be a valid python dictionary with two customary keys:
        - loss: Specify a numeric evaluation metric to be minimized
        - status: Just use STATUS_OK and see hyperopt documentation if not feasible
    The last one is optional, though recommended, namely:
        - model: specify the model just created so that we can later use it again.
    """
    
    ## Building the Model
    pmts = int(7) # number of PMTs
    
    model = Sequential()
    model.add(Dense(64, input_shape=(pmts,)))
    model.add(Activation({{choice(['relu', 'sigmoid', 'linear', 'softplus', 'tanh'])}}))
    #model.add(Dropout(0.1))
    model.add(Dense(64))
    model.add(Activation({{choice(['relu', 'sigmoid', 'linear', 'softplus', 'tanh'])}}))
    #model.add(Dropout(0.1))
    model.add(Dense(2))  # x and y
    model.add(Activation('linear'))
    
    ## Compiling the Model
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['mean_squared_error']
             )
    
    ## Training the Model
    result = model.fit(train_feature, train_label,
                       epochs=30,
                       batch_size=128,
                       validation_split=0.2,
                       verbose=2
                      )
    
    ## Get the lowest validation loss of the training epochs
    validation_loss = np.amin(result.history['val_loss']) 
    print('Best validation loss of epoch:', validation_loss)
    return {'loss': validation_loss, 'status': STATUS_OK, 'model': model}

In [6]:
#############################################
## Optimize the NN
## Make max_evals sufficiently large to cover
## entire paramater space.
#############################################

max_ev = 70

if b_GC == True:
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=max_ev,
                                          trials=Trials(),
                                          eval_space = True,
                                          notebook_name='gdrive/My Drive/Colab Notebooks/XeBRA_Position_Reconstruction_Optimize_NN',
                                          keep_temp=False
                                         )
else: 
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=max_ev,
                                          trials=Trials(),
                                          eval_space = True,
                                          notebook_name='XeBRA_Position_Reconstruction_Optimize_NN',
                                          keep_temp=False
                                         )

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from keras.layers.core import Dense, Dropout, Activation
except:
    pass

try:
    from keras.models import Sequential
except:
    pass

try:
    from keras.utils import np_utils
except:
    pass

try:
    from keras.optimizers import SGD, Adam
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import os
except:
    pass

try:
    import glob
except:
    pass

try:
    import uproot
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    from google.colab import drive
except:
    pass

try:
    import concurrent.futures
except:
    pass

>>> Hyperas search space:

def get_space():
    return {
        'Activation': hp.choice('Activation', ['relu', 'sigmoid', 'linear', 'soft

/home/ab602/anaconda3/lib/python3.7/site-packages/uproot/tree.py:116: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  index = awkward.numpy.frombuffer(arrays.index.data, dtype=arrays.index.dtype)


  0%|          | 0/30 [00:00<?, ?it/s, best loss: ?]


W0813 16:45:20.006234 140685427783488 deprecation_wrapper.py:119] From /home/ab602/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.


W0813 16:45:20.017384 140685427783488 deprecation_wrapper.py:119] From /home/ab602/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.


W0813 16:45:20.019028 140685427783488 deprecation_wrapper.py:119] From /home/ab602/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.


W0813 16:45:20.044433 140685427783488 deprecation_wrapper.py:119] From /home/ab602/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.


W0813 1

Train on 720000 samples, validate on 180000 samples 
Epoch 1/30                                          
 - 9s - loss: 0.0016 - mean_squared_error: 0.0016 - val_loss: 5.5428e-04 - val_mean_squared_error: 5.5428e-04

Epoch 2/30                                          
 - 7s - loss: 5.1556e-04 - mean_squared_error: 5.1556e-04 - val_loss: 4.8266e-04 - val_mean_squared_error: 4.8266e-04

Epoch 3/30                                          
 - 7s - loss: 4.8048e-04 - mean_squared_error: 4.8048e-04 - val_loss: 4.7972e-04 - val_mean_squared_error: 4.7972e-04

Epoch 4/30                                          
 - 9s - loss: 4.6619e-04 - mean_squared_error: 4.6619e-04 - val_loss: 4.9446e-04 - val_mean_squared_error: 4.9446e-04

Epoch 5/30                                          
 - 9s - loss: 4.5146e-04 - mean_squared_error: 4.5146e-04 - val_loss: 4.2937e-04 - val_mean_squared_error: 4.2937e-04

Epoch 6/30                                          
 - 9s - loss: 4.4039e-04 - mean_squared_er

...

Epoch 18/30                                                                         
 - 16s - loss: 4.1682e-04 - mean_squared_error: 4.1682e-04 - val_loss: 3.9368e-04 - val_mean_squared_error: 3.9368e-04

Epoch 19/30                                                                         
 - 16s - loss: 4.1683e-04 - mean_squared_error: 4.1683e-04 - val_loss: 4.2971e-04 - val_mean_squared_error: 4.2971e-04

Epoch 20/30                                                                         
 - 15s - loss: 4.1498e-04 - mean_squared_error: 4.1498e-04 - val_loss: 4.0751e-04 - val_mean_squared_error: 4.0751e-04

Epoch 21/30                                                                         
 - 16s - loss: 4.1522e-04 - mean_squared_error: 4.1522e-04 - val_loss: 4.2450e-04 - val_mean_squared_error: 4.2450e-04

Epoch 22/30                                                                         
 - 16s - loss: 4.1151e-04 - mean_squared_error: 4.1151e-04 - val_loss: 4.2911e-04 - val_mean_sq

 - 13s - loss: 3.9792e-04 - mean_squared_error: 3.9792e-04 - val_loss: 3.9802e-04 - val_mean_squared_error: 3.9802e-04

Epoch 27/30                                                                         
 - 14s - loss: 3.9764e-04 - mean_squared_error: 3.9764e-04 - val_loss: 3.9843e-04 - val_mean_squared_error: 3.9843e-04

Epoch 28/30                                                                         
 - 14s - loss: 3.9697e-04 - mean_squared_error: 3.9697e-04 - val_loss: 4.0663e-04 - val_mean_squared_error: 4.0663e-04

Epoch 29/30                                                                         
 - 13s - loss: 3.9704e-04 - mean_squared_error: 3.9704e-04 - val_loss: 3.9289e-04 - val_mean_squared_error: 3.9289e-04

Epoch 30/30                                                                         
 - 13s - loss: 3.9678e-04 - mean_squared_error: 3.9678e-04 - val_loss: 4.0293e-04 - val_mean_squared_error: 4.0293e-04

Best validation loss of epoch:                              

Epoch 4/30                                                                          
 - 15s - loss: 4.0038e-04 - mean_squared_error: 4.0038e-04 - val_loss: 4.0849e-04 - val_mean_squared_error: 4.0849e-04

Epoch 5/30                                                                          
 - 16s - loss: 3.9710e-04 - mean_squared_error: 3.9710e-04 - val_loss: 3.8418e-04 - val_mean_squared_error: 3.8418e-04

Epoch 6/30                                                                          
 - 16s - loss: 3.9289e-04 - mean_squared_error: 3.9289e-04 - val_loss: 4.3018e-04 - val_mean_squared_error: 4.3018e-04

Epoch 7/30                                                                          
 - 15s - loss: 3.9086e-04 - mean_squared_error: 3.9086e-04 - val_loss: 4.0998e-04 - val_mean_squared_error: 4.0998e-04

Epoch 8/30                                                                          
 - 14s - loss: 3.8840e-04 - mean_squared_error: 3.8840e-04 - val_loss: 3.8994e-04 - val_mean_sq

 - 12s - loss: 3.8245e-04 - mean_squared_error: 3.8245e-04 - val_loss: 3.8339e-04 - val_mean_squared_error: 3.8339e-04

Epoch 13/30                                                                         
 - 12s - loss: 3.8191e-04 - mean_squared_error: 3.8191e-04 - val_loss: 3.8680e-04 - val_mean_squared_error: 3.8680e-04

Epoch 14/30                                                                         
 - 13s - loss: 3.8096e-04 - mean_squared_error: 3.8096e-04 - val_loss: 3.7129e-04 - val_mean_squared_error: 3.7129e-04

Epoch 15/30                                                                         
 - 13s - loss: 3.8015e-04 - mean_squared_error: 3.8015e-04 - val_loss: 3.8997e-04 - val_mean_squared_error: 3.8997e-04

Epoch 16/30                                                                         
 - 13s - loss: 3.7972e-04 - mean_squared_error: 3.7972e-04 - val_loss: 3.8075e-04 - val_mean_squared_error: 3.8075e-04

Epoch 17/30                                                 

Epoch 21/30                                                                         
 - 16s - loss: 4.0677e-04 - mean_squared_error: 4.0677e-04 - val_loss: 4.5646e-04 - val_mean_squared_error: 4.5646e-04

Epoch 22/30                                                                         
 - 16s - loss: 4.0587e-04 - mean_squared_error: 4.0587e-04 - val_loss: 3.9644e-04 - val_mean_squared_error: 3.9644e-04

Epoch 23/30                                                                         
 - 16s - loss: 4.0532e-04 - mean_squared_error: 4.0532e-04 - val_loss: 4.0696e-04 - val_mean_squared_error: 4.0696e-04

Epoch 24/30                                                                         
 - 16s - loss: 4.0495e-04 - mean_squared_error: 4.0495e-04 - val_loss: 4.1069e-04 - val_mean_squared_error: 4.1069e-04

Epoch 25/30                                                                         
 - 15s - loss: 4.0451e-04 - mean_squared_error: 4.0451e-04 - val_loss: 4.1078e-04 - val_mean_sq

In [7]:
X_train, Y_train, X_test, Y_test = data()
print("Evalutation of best performing model:")
print(best_model.evaluate(X_test, Y_test))
print("Best performing model chosen hyper-parameters:")
print(best_run)

best_model.save('best_model.h5')

/home/ab602/anaconda3/lib/python3.7/site-packages/uproot/tree.py:116: FutureWarning: Int64Index.data is deprecated and will be removed in a future version
  index = awkward.numpy.frombuffer(arrays.index.data, dtype=arrays.index.dtype)


Evalutation of best performing model:
100000/100000 [==============================] - ETA:  - 2s 15us/step
[0.00037690408617956566, 0.00037690408617956566]
Best performing model chosen hyper-parameters:
{'Activation': 'relu', 'Activation_1': 'relu'}


In [8]:
## Save optimization results to txt file

fh = open("Optimization_results.txt", "w") 

fh.write("Evalutation of best performing model:\n")
fh.write(str(best_model.evaluate(X_test, Y_test))+"\n")
fh.write("Best performing model chosen hyper-parameters:\n")
fh.write(str(best_run)+"\n")
 
fh.close() 

100000/100000 [==============================] - 1s 14us/step
